1. 데이터베이스에서 사용 가능한 테이블과 스키마 가져오기.
1. 질문과 관련된 테이블을 LLM이 결정
1. 해당 테이블의 스키마 확인하기
1. 질문과 스키마의 정보를 기반으로 쿼리 생성
1. LLM을 사용하여 흔히 발생하는 오류가 있는지 SQL 확인
1. DB에서 SQL을 실행하고 결과 확인
1. DB에서 에러 발생 시, 수정 후 다시 확인
1. 결과를 바탕으로 답변을 작성하기

In [ ]:
# %pip install -q langgraph sqlalchemy psycopg2-binary

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os
from langchain_community.utilities import SQLDatabase

DB_URI = os.environ.get('DB_URI')
db = SQLDatabase.from_uri(DB_URI)

print(db.get_usable_table_names())
print(db.run('SELECT * FROM sales LIMIT 5;'))

In [ ]:
# LLM 초기화
from langchain_openai import ChatOpenAI

model = ChatOpenAI(name='gpt-4.1-mini')

In [12]:
# Agent용 Tool 만들기
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)  # Tool이 여러개 모여 있는 구조

In [ ]:
# Agent 만들기
from langchain.agents import create_agent

dialect = db.dialect
top_k = 5

system_prompt = f"""
    You are an agent designed to interact with a SQL database.
    Given an input question, create a syntactically correct {dialect} query to run,
    then look at the results of the query and return the answer. Unless the user
    specifies a specific number of examples they wish to obtain, always limit your
    query to at most {top_k} results.

    You can order the results by a relevant column to return the most interesting
    examples in the database. Never query for all the columns from a specific table,
    only ask for the relevant columns given the question.

    You MUST double check your query before executing it. If you get an error while
    executing a query, rewrite the query and try again.

    DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
    database.

    To start you should ALWAYS look at the tables in the database to see what you
    can query. Do NOT skip this step.

    Then you should query the schema of the most relevant tables.
    """

agent = create_agent(model, toolkit.get_tools(), system_prompt=system_prompt)   # 변수에 저장 안 하면 LangGraph 형태가(이미지) 나옴

## Human In  the Loop

In [18]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# 사람이 개입할 수 있는 middleware 생성
agent = create_agent(
    model,
    toolkit.get_tools(),
    system_prompt=system_prompt,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={'sql_db_query': True},    # 언제 중단할지
            description_prefix='Tool 실행 전에 승인을 기다림' 
        )
    ],
    checkpointer=InMemorySaver()        # 일시정지 후 어디서 재시작할 지 기억해야 하기 때문에 사용
)

In [ ]:
from langgraph.types import Command

question = '2월에 가장 많이 팔린 물건 3개와, 해당 물건들의 토요일 일요일 평균 매출액'

config = {'configurable': {'thread_id': '123456'}}

for event in agent.stream(
    {'messages': [{'role': 'user', 'content': question}]},
    stream_mode='values',
    config=config,
):
    if "__interrupt__" in event: 
        print("INTERRUPTED:") 
        interrupt = event["__interrupt__"][0] 
        for request in interrupt.value["action_requests"]: 
            print(request["description"]) 
    elif "messages" in event:
        event["messages"][-1].pretty_print()
    else:
        pass

print('-----------------------------------------------------------------')

# 사용자가 화면에서 버튼을 클릭했다는 전제하에 잠깐 멈췄다가 다시 실행하는 코드
for step in agent.stream(
    Command(resume={"decisions": [{"type": "approve"}]}), 
    config,
    stream_mode="values",
):
    if "messages" in step:
        step["messages"][-1].pretty_print()
    elif "__interrupt__" in step:
        print("INTERRUPTED:")
        interrupt = step["__interrupt__"][0]
        for request in interrupt.value["action_requests"]:
            print(request["description"])
    else:
        pass

================================ Human Message =================================

2월에 가장 많이 팔린 물건 3개와, 해당 물건들의 토요일 일요일 평균 매출액
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_Jhs9DZL6QkWz1Z3msDkzLDSn)
 Call ID: call_Jhs9DZL6QkWz1Z3msDkzLDSn
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

courses, customers, dt_demo, inventory, lotto_draws, members, sales, sample, students, students_courses, test_n8n_chat_histories
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_QyDczjw6HKrrkavndPHwNRHz)
 Call ID: call_QyDczjw6HKrrkavndPHwNRHz
  Args:
    table_names: sales, dt_demo
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE dt_demo (
	id INTEGER GENERATED ALWAYS AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 2147483